In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/Weather_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,2020-09-13 20:35:30,-21.21,-159.78,75.20,73,97,9.17,overcast clouds
1,1,Lebu,CL,2020-09-13 20:35:31,-37.62,-73.65,51.17,68,0,18.21,clear sky
2,2,Le Vauclin,MQ,2020-09-13 20:35:31,14.55,-60.84,81.00,88,32,3.36,mist
3,3,East London,ZA,2020-09-13 20:35:31,-33.02,27.91,63.30,57,69,8.39,broken clouds
4,4,Rikitea,PF,2020-09-13 20:35:31,-23.12,-134.97,73.58,84,0,13.62,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,2020-09-13 20:35:30,-21.21,-159.78,75.20,73,97,9.17,overcast clouds
2,2,Le Vauclin,MQ,2020-09-13 20:35:31,14.55,-60.84,81.00,88,32,3.36,mist
4,4,Rikitea,PF,2020-09-13 20:35:31,-23.12,-134.97,73.58,84,0,13.62,clear sky
5,5,Albany,US,2020-09-13 20:35:31,42.60,-73.97,73.99,65,65,6.58,broken clouds
9,9,Ribeira Grande,PT,2020-09-13 20:35:32,38.52,-28.70,73.40,83,40,14.99,scattered clouds
14,14,Sao Joao Del Rei,BR,2020-09-13 20:35:32,-21.14,-44.26,78.80,27,0,8.05,clear sky
23,23,Gouyave,GD,2020-09-13 20:35:33,12.16,-61.73,89.60,75,40,17.22,scattered clouds
30,30,Lincoln,US,2020-09-13 20:33:33,40.80,-96.67,84.00,34,1,3.80,clear sky
33,33,Qui Nhon,VN,2020-09-13 20:35:36,13.77,109.23,78.33,85,95,4.97,overcast clouds
35,35,Bubaque,GW,2020-09-13 20:35:36,11.28,-15.83,80.85,87,52,18.30,broken clouds


In [14]:
# 4a. Determine if there are any empty rows.
np.where(pd.isnull(preferred_cities_df))
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
0,0,Avarua,CK,2020-09-13 20:35:30,-21.21,-159.78,75.20,73,97,9.17,overcast clouds,
2,2,Le Vauclin,MQ,2020-09-13 20:35:31,14.55,-60.84,81.00,88,32,3.36,mist,
4,4,Rikitea,PF,2020-09-13 20:35:31,-23.12,-134.97,73.58,84,0,13.62,clear sky,
5,5,Albany,US,2020-09-13 20:35:31,42.60,-73.97,73.99,65,65,6.58,broken clouds,
9,9,Ribeira Grande,PT,2020-09-13 20:35:32,38.52,-28.70,73.40,83,40,14.99,scattered clouds,
...,...,...,...,...,...,...,...,...,...,...,...,...
684,684,Kelowna,CA,2020-09-13 20:35:04,49.88,-119.49,72.00,43,75,3.36,haze,
687,687,Quang Ngai,VN,2020-09-13 20:37:29,15.12,108.80,75.99,89,100,3.62,overcast clouds,
688,688,Wanning,CN,2020-09-13 20:37:30,18.80,110.40,79.45,81,83,2.59,broken clouds,
689,689,Sao Pedro,BR,2020-09-13 20:37:30,-22.55,-47.91,83.37,30,0,3.38,clear sky,


In [16]:
pd.isna(preferred_cities_df['City'])

0      False
2      False
4      False
5      False
9      False
       ...  
684    False
687    False
688    False
689    False
691    False
Name: City, Length: 324, dtype: bool

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df.loc[:,("City", "Country", "Max Temp", "Current Description", "Lat", "Lng")].copy()

# 5b. Create a new column "Hotel Name"
clean_df["Hotel Name"] = ""
clean_df.head(10)

/Users/brianahn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
0,0,Avarua,CK,2020-09-13 20:35:30,-21.21,-159.78,75.20,73,97,9.17,overcast clouds,
2,2,Le Vauclin,MQ,2020-09-13 20:35:31,14.55,-60.84,81.00,88,32,3.36,mist,
4,4,Rikitea,PF,2020-09-13 20:35:31,-23.12,-134.97,73.58,84,0,13.62,clear sky,
5,5,Albany,US,2020-09-13 20:35:31,42.60,-73.97,73.99,65,65,6.58,broken clouds,
9,9,Ribeira Grande,PT,2020-09-13 20:35:32,38.52,-28.70,73.40,83,40,14.99,scattered clouds,
14,14,Sao Joao Del Rei,BR,2020-09-13 20:35:32,-21.14,-44.26,78.80,27,0,8.05,clear sky,
23,23,Gouyave,GD,2020-09-13 20:35:33,12.16,-61.73,89.60,75,40,17.22,scattered clouds,
30,30,Lincoln,US,2020-09-13 20:33:33,40.80,-96.67,84.00,34,1,3.80,clear sky,
33,33,Qui Nhon,VN,2020-09-13 20:35:36,13.77,109.23,78.33,85,95,4.97,overcast clouds,
35,35,Bubaque,GW,2020-09-13 20:35:36,11.28,-15.83,80.85,87,52,18.30,broken clouds,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [19]:
# 6b. Iterate through the hotel DataFrame.
for index, row in clean_df.iterrows():
    ## 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary 
    # as values to the location key.
    params["location"] = f"{lat},{lng}"

    ## 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
   # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    try:
   # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        clean_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
    

/Users/brianahn/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
0,0,Avarua,CK,2020-09-13 20:35:30,-21.21,-159.78,75.20,73,97,9.17,overcast clouds,Paradise Inn
2,2,Le Vauclin,MQ,2020-09-13 20:35:31,14.55,-60.84,81.00,88,32,3.36,mist,Hôtel Cap Macabou
4,4,Rikitea,PF,2020-09-13 20:35:31,-23.12,-134.97,73.58,84,0,13.62,clear sky,Pension Maro'i
5,5,Albany,US,2020-09-13 20:35:31,42.60,-73.97,73.99,65,65,6.58,broken clouds,
9,9,Ribeira Grande,PT,2020-09-13 20:35:32,38.52,-28.70,73.40,83,40,14.99,scattered clouds,Quinta da Meia Eira
...,...,...,...,...,...,...,...,...,...,...,...,...
684,684,Kelowna,CA,2020-09-13 20:35:04,49.88,-119.49,72.00,43,75,3.36,haze,Delta Hotels by Marriott Grand Okanagan Resort
687,687,Quang Ngai,VN,2020-09-13 20:37:29,15.12,108.80,75.99,89,100,3.62,overcast clouds,Central Hotel
688,688,Wanning,CN,2020-09-13 20:37:30,18.80,110.40,79.45,81,83,2.59,broken clouds,Paradise Hotel
689,689,Sao Pedro,BR,2020-09-13 20:37:30,-22.55,-47.91,83.37,30,0,3.38,clear sky,Hotel Fazenda Samambaia


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_df

/Users/brianahn/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
0,0,Avarua,CK,2020-09-13 20:35:30,-21.21,-159.78,75.20,73,97,9.17,overcast clouds,Paradise Inn
2,2,Le Vauclin,MQ,2020-09-13 20:35:31,14.55,-60.84,81.00,88,32,3.36,mist,Hôtel Cap Macabou
4,4,Rikitea,PF,2020-09-13 20:35:31,-23.12,-134.97,73.58,84,0,13.62,clear sky,Pension Maro'i
5,5,Albany,US,2020-09-13 20:35:31,42.60,-73.97,73.99,65,65,6.58,broken clouds,NaN
9,9,Ribeira Grande,PT,2020-09-13 20:35:32,38.52,-28.70,73.40,83,40,14.99,scattered clouds,Quinta da Meia Eira
...,...,...,...,...,...,...,...,...,...,...,...,...
684,684,Kelowna,CA,2020-09-13 20:35:04,49.88,-119.49,72.00,43,75,3.36,haze,Delta Hotels by Marriott Grand Okanagan Resort
687,687,Quang Ngai,VN,2020-09-13 20:37:29,15.12,108.80,75.99,89,100,3.62,overcast clouds,Central Hotel
688,688,Wanning,CN,2020-09-13 20:37:30,18.80,110.40,79.45,81,83,2.59,broken clouds,Paradise Hotel
689,689,Sao Pedro,BR,2020-09-13 20:37:30,-22.55,-47.91,83.37,30,0,3.38,clear sky,Hotel Fazenda Samambaia


In [22]:
clean_df.dropna(subset=['Hotel Name'], inplace=True)
clean_df

/Users/brianahn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
0,0,Avarua,CK,2020-09-13 20:35:30,-21.21,-159.78,75.20,73,97,9.17,overcast clouds,Paradise Inn
2,2,Le Vauclin,MQ,2020-09-13 20:35:31,14.55,-60.84,81.00,88,32,3.36,mist,Hôtel Cap Macabou
4,4,Rikitea,PF,2020-09-13 20:35:31,-23.12,-134.97,73.58,84,0,13.62,clear sky,Pension Maro'i
9,9,Ribeira Grande,PT,2020-09-13 20:35:32,38.52,-28.70,73.40,83,40,14.99,scattered clouds,Quinta da Meia Eira
14,14,Sao Joao Del Rei,BR,2020-09-13 20:35:32,-21.14,-44.26,78.80,27,0,8.05,clear sky,Solara Hotel
...,...,...,...,...,...,...,...,...,...,...,...,...
684,684,Kelowna,CA,2020-09-13 20:35:04,49.88,-119.49,72.00,43,75,3.36,haze,Delta Hotels by Marriott Grand Okanagan Resort
687,687,Quang Ngai,VN,2020-09-13 20:37:29,15.12,108.80,75.99,89,100,3.62,overcast clouds,Central Hotel
688,688,Wanning,CN,2020-09-13 20:37:30,18.80,110.40,79.45,81,83,2.59,broken clouds,Paradise Hotel
689,689,Sao Pedro,BR,2020-09-13 20:37:30,-22.55,-47.91,83.37,30,0,3.38,clear sky,Hotel Fazenda Samambaia


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/Hotel_List.csv"
clean_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [28]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]


In [30]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_df[["Lat", "Lng"]]
max_temp = clean_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))